In [1]:
import pandas as pd
import numpy as np
import warnings
import eventstudy as es
# from utils import XD
from scipy.stats import wilcoxon
import pymannkendall as mk
from tqdm import tqdm
import itertools
import eventstudy
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [2]:
# read data
cn = pd.read_csv("../../1223/data/concentration/2020_20230814.csv").dropna()
q = pd.read_csv("../../1223/data/quote/2020_20230814.csv").dropna()
rf = pd.read_csv("../../1223/data/rf/rf.csv").dropna().rename(columns={'日期':'date'})
df = pd.merge(q, cn, on=['日期', '股號'], how='left')
df = df.groupby('股號').apply(lambda x: x.sort_values('日期')).reset_index(drop=True)

In [3]:
import yfinance as yf

# 設定股票代碼，台灣加權指數的代碼是 ^TWII
stock_code = "^TWII"

# 設定開始日期和結束日期
start_date = "2020-01-01"
end_date = "2023-12-31"  # 你可以調整結束日期為當前日期或任何你需要的日期

# 使用yfinance獲取股價資訊
twii_data = yf.download(stock_code, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [4]:
twii_data['date'] =  pd.to_datetime(twii_data.index)
twii_data['ret'] = twii_data['Adj Close'].pct_change()

In [5]:
# mutate col
df = pd.merge(q, cn, on=['日期', '股號'], how='left')
df = df.groupby('股號').apply(lambda x: x.sort_values('日期')).reset_index(drop=True)


# 量起
df['成交量_1'] = df.groupby('股號')['成交量'].shift(-1)

def divide_two_cols(df_sub):
    df_sub['volume_delta_1'] = df_sub['成交量_1'] / df_sub['成交量']
    return df_sub

df = df.groupby('股號').apply(divide_two_cols)


# 價揚
# shift 1 假設是成交量出現異常後的下一根進場
df['ret'] = df.groupby('股號')['收盤價'].pct_change()
df['ret_2'] = df.groupby('股號')['ret'].shift(-1)
df = df.dropna()

In [6]:
def create_ret_df(df):

    uq_id = df['股號'].unique()
    res_by_ticker = []
    res = None

    for ticker_id in tqdm(uq_id[:len(uq_id)]):
        ticker_df = df[df['股號']==ticker_id].reset_index(drop=True)

        date = ticker_df['日期'].tolist()
        ret = ticker_df['ret'].tolist()

        ticker_ret_df = pd.DataFrame(list(zip(date, ret)),
               columns =['date', f'{ticker_id}'])
        
        res_by_ticker.append(ticker_ret_df)
    for i in tqdm(range(len(res_by_ticker))):
        if i == 0:
            res = res_by_ticker[i]

        else:
            res = pd.merge(res, res_by_ticker[i], how='left', on='date')

    return res


In [7]:
return_df = create_ret_df(df)

100%|██████████| 1350/1350 [01:25<00:00, 15.82it/s]


In [8]:
rf["date"] = pd.to_datetime(rf['date'])
return_df["date"] =  pd.to_datetime(return_df["date"])

In [28]:
main_df_4analysis = pd.merge(return_df, rf, how='left', on='date')
main_df_4analysis = pd.merge(main_df_4analysis, twii_data, how='left', on='date')
main_df_4analysis = main_df_4analysis.dropna(subset=['1101', 'ret'])

In [30]:
# fama df
famadf = main_df_4analysis.iloc[:,[main_df_4analysis.columns.get_loc(c) for c in ['date', '收市', 'ret']]].rename(columns={'收市':'RF', 'ret':'MktRt'})
famadf['Mkt-RF'] = famadf['MktRt'] - famadf['RF']
famadf['SMB'] = 0
famadf['HML'] = 0
# print(famadf.dropna())
# famadf = famadf.dropna(subset=['1101', 'toy'])
famadf.to_csv('./df/fama.csv')
print(famadf)

# return df
returns_df = main_df_4analysis.iloc[:,[main_df_4analysis.columns.get_loc(c) for c in ['date', '1101']]] 
print(returns_df)
returns_df.to_csv('./df/returns.csv')

          date     RF     MktRt    Mkt-RF  SMB  HML
0   2020-01-03  0.632  0.000822 -0.631178    0    0
1   2020-01-06  0.623 -0.012970 -0.635970    0    0
2   2020-01-07  0.623 -0.006110 -0.629110    0    0
3   2020-01-08  0.623 -0.005321 -0.628321    0    0
4   2020-01-09  0.627  0.012992 -0.614008    0    0
..         ...    ...       ...       ...  ...  ...
871 2023-08-07  1.163  0.009043 -1.153957    0    0
872 2023-08-08  1.164 -0.006998 -1.170998    0    0
873 2023-08-09  1.157 -0.000363 -1.157363    0    0
874 2023-08-10  1.157 -0.014003 -1.171003    0    0
875 2023-08-11  1.165 -0.002011 -1.167011    0    0

[875 rows x 6 columns]
          date      1101
0   2020-01-03 -0.003401
1   2020-01-06 -0.011377
2   2020-01-07  0.003452
3   2020-01-08 -0.004587
4   2020-01-09  0.001152
..         ...       ...
871 2023-08-07  0.008032
872 2023-08-08 -0.014608
873 2023-08-09  0.001348
874 2023-08-10 -0.008075
875 2023-08-11  0.014925

[875 rows x 2 columns]


In [31]:
df_1101 = df[df['股號']==1101]


vol_increase_id = df_1101[(df_1101['成交量']>500)&(df_1101['volume_delta_1']>2)].index
df_1101['indicator'] = df_1101.index.isin(vol_increase_id).astype(int)
df_1101['event1'] = (df_1101['indicator'].shift(1) == 0) & (df_1101['indicator'] == 1)

event_date = df_1101['日期'][vol_increase_id].to_list()
event_date[5]

'2020-06-18'

In [32]:
es.Single.import_returns(path='./df/returns.csv')
es.Single.import_FamaFrench(path='./df/fama.csv', date_format = '%Y-%m-%d %H:%M:%S.%f')

In [36]:
# try:
event = es.Single.FamaFrench_3factor(
    security_ticker = '1101',
    event_date = np.datetime64('2020-04-13'),
    event_window = (-2,+10), 
    estimation_size = 10, # 注意這個
    buffer_size = 30
)
# except:
    # print()

In [49]:
test = event.results(decimals=[3,5,3,5,2,2])

In [53]:
test['EventDate'] =  np.datetime64('2020-06-18')
test['symbol'] = '1101'

In [54]:
test

,AR,Std. E. AR,CAR,Std. E. CAR,T-stat,P-value,EventDate,symbol
-2,-0.010,0.01012,-0.01,0.01012,-0.97,0.36,2020-06-18,1101
-1,-0.016,0.01012,-0.025,0.01431,-1.78,0.11,2020-06-18,1101
0,-0.020,0.01012,-0.045 **,0.01753,-2.59,0.03,2020-06-18,1101
1,-0.019,0.01012,-0.064 **,0.02024,-3.17,0.01,2020-06-18,1101
2,-0.007,0.01012,-0.071 **,0.02263,-3.16,0.01,2020-06-18,1101
3,-0.020,0.01012,-0.091 **,0.02479,-3.67,0.01,2020-06-18,1101
4,-0.013,0.01012,-0.104 ***,0.02678,-3.87,0.00,2020-06-18,1101
5,-0.020,0.01012,-0.123 ***,0.02863,-4.30,0.00,2020-06-18,1101
6,-0.015,0.01012,-0.139 ***,0.03036,-4.56,0.00,2020-06-18,1101
7,-0.010,0.01012,-0.148 ***,0.03200,-4.63,0.00,2020-06-18,1101
